In [1]:
using LinearAlgebra
using CUDA
using Knet

In [2]:
LARGE_NUM = 1e9
temperature = 1.0
weights = 1.0

1.0

In [3]:
_atype = CUDA.functional() ? KnetArray{Float32} : Array{Float32}

KnetArray{Float32,N} where N

In [21]:
z = Array([1.0 2.0
        3.0 -2.0
        1.0 5.0
        1.0 0.75
        2.8 -1.75
        1.0 4.7])

6×2 Array{Float64,2}:
 1.0   2.0
 3.0  -2.0
 1.0   5.0
 1.0   0.75
 2.8  -1.75
 1.0   4.7

In [13]:
z

6×2 Array{Float64,2}:
 1.0   2.0
 3.0  -2.0
 1.0   5.0
 1.0   0.75
 2.8  -1.75
 1.0   4.7

In [14]:
norm_z = convert(_atype, mapslices(norm, Array(z), dims=[2]))

6×1 KnetArray{Float32,2}:
 2.236068
 3.6055512
 5.0990195
 1.25
 3.3018935
 4.8052053

In [15]:
norm_z = zeros(size(z)[1])
for i in 1:size(z)[1]
    v =norm(getindex(z,i,:))
    setindex!(norm_z, v, i)
end
norm_z

6-element Array{Float64,1}:
 2.23606797749979
 3.605551275463989
 5.0990195135927845
 1.25
 3.3018933962198114
 4.805205510693586

In [16]:
norm_z = sqrt.(sum(abs2,z,dims=2))
norm_z

6×1 Array{Float64,2}:
 2.23606797749979
 3.605551275463989
 5.0990195135927845
 1.25
 3.3018933962198114
 4.805205510693586

In [35]:
norm_z = convert(_atype,norm_z)
z = convert(_atype,z)

6×2 KnetArray{Float32,2}:
 1.0   2.0
 3.0  -2.0
 1.0   5.0
 1.0   0.75
 2.8  -1.75
 1.0   4.7

In [36]:
zx = z ./ norm_z

6×2 KnetArray{Float32,2}:
 0.447214   0.894427
 0.83205   -0.5547
 0.196116   0.980581
 0.8        0.6
 0.847998  -0.529999
 0.208108   0.978106

In [24]:
bs = size(zx)[1]÷2

3

In [38]:
z1 = zx[1:bs,:]

3×2 KnetArray{Float32,2}:
 0.447214   0.894427
 0.83205   -0.5547
 0.196116   0.980581

In [39]:
z2 = zx[bs+1:bs*2,:]

3×2 KnetArray{Float32,2}:
 0.8        0.6
 0.847998  -0.529999
 0.208108   0.978106

In [40]:
labels = convert(_atype, Array{Float64}(I, bs, bs*2))

3×6 KnetArray{Float32,2}:
 1.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0

In [41]:
mask = convert(_atype, Array{Float64}(I, bs, bs)*LARGE_NUM)

3×3 KnetArray{Float32,2}:
 1.0f9  0.0    0.0
 0.0    1.0f9  0.0
 0.0    0.0    1.0f9

In [42]:
logits_aa = z1*transpose(z1)-mask

3×3 KnetArray{Float32,2}:
 -1.0f9     -0.124035   0.964764
 -0.124035  -1.0f9     -0.38075
  0.964764  -0.38075   -1.0f9

In [43]:
logits_bb = z2*transpose(z2)-mask

3×3 KnetArray{Float32,2}:
 -1.0f9      0.360399   0.75335
  0.360399  -1.0f9     -0.34192
  0.75335   -0.34192   -1.0f9

In [44]:
logits_ab = z1*transpose(z2)

3×3 KnetArray{Float32,2}:
 0.894427  -0.0948091   0.967913
 0.33282    0.999568   -0.369399
 0.745241  -0.353401    0.999925

In [45]:
logits_ba = z2*transpose(z1)

3×3 KnetArray{Float32,2}:
  0.894427    0.33282    0.745241
 -0.0948091   0.999568  -0.353401
  0.967913   -0.369399   0.999925

In [46]:
[logits_ab logits_aa]

3×6 KnetArray{Float32,2}:
 0.894427  -0.0948091   0.967913  -1.0f9     -0.124035   0.964764
 0.33282    0.999568   -0.369399  -0.124035  -1.0f9     -0.38075
 0.745241  -0.353401    0.999925   0.964764  -0.38075   -1.0f9

In [47]:
softmax([logits_ab logits_aa], dims=2)

3×6 KnetArray{Float32,2}:
 0.25759   0.0957877  0.277232  0.0       0.0930287  0.276361
 0.218984  0.426557   0.108503  0.138677  0.0        0.107279
 0.238482  0.0794916  0.307654  0.297025  0.077347   0.0

In [48]:
logsoftmax([logits_ab logits_aa], dims=2)

3×6 KnetArray{Float32,2}:
 -1.35638  -2.34562   -1.2829   -1.0f9    -2.37485  -1.28605
 -1.51876  -0.852008  -2.22098  -1.97561  -1.0f9    -2.23233
 -1.43346  -2.5321    -1.17878  -1.21394  -2.55945  -1.0f9

In [49]:
-labels.*logsoftmax([logits_ab logits_aa], dims=2)

3×6 KnetArray{Float32,2}:
 1.35638  0.0       0.0      0.0  0.0  0.0
 0.0      0.852008  0.0      0.0  0.0  0.0
 0.0      0.0       1.17878  0.0  0.0  0.0

In [51]:
loss_a = sum(-labels.*logsoftmax([logits_ab logits_aa], dims=2))/bs

1.129057f0

In [52]:
loss_b = sum(-labels.*logsoftmax([logits_ba logits_bb], dims=2))/bs

1.1363438f0

In [53]:
loss = loss_a + loss_b

2.265401f0